In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, GlobalMaxPooling2D, Dense, Input, Concatenate
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set variables and hyperparameters

input_shape = (224, 224, 3)
num_classes = 16
output_activation = 'softmax'
activation_function = 'relu'
optimizer = Adam(learning_rate=0.001)
batch_size = 32
epochs = 50
loss_function = 'categorical_crossentropy'

train_dir = '/content/drive/MyDrive/Research/Data/Dataset/train'
val_dir = '/content/drive/MyDrive/Research/Data/Dataset/validate'
test_dir = '/content/drive/MyDrive/Research/Data/Dataset/test'

In [ ]:
# Prepare the dataset
def preprocess_data(train_dir, val_dir, test_dir, input_size=(224, 224), batch_size=batch_size):
    datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                 height_shift_range=0.2, horizontal_flip=True)

    train_generator = datagen.flow_from_directory(train_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical')
    val_generator = datagen.flow_from_directory(val_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical')
    test_generator = datagen.flow_from_directory(test_dir, target_size=input_size, batch_size=batch_size, class_mode='categorical')

    return train_generator, val_generator, test_generator

In [ ]:
# Define the model architecture
def create_model(input_shape, num_classes):
    # Input layer
    inputs = Input(shape=input_shape)

    # First convolutional level
    conv1 = Conv2D(32, (3, 3), activation=activation_function)(inputs)
    gmpl1 = GlobalMaxPooling2D()(conv1)
    fc1 = Dense(64, activation=activation_function)(gmpl1)

    # Second convolutional level
    conv2 = Conv2D(64, (3, 3), activation=activation_function)(inputs)
    gmp2 = GlobalMaxPooling2D()(conv2)
    fc2 = Dense(64, activation=activation_function)(gmp2)

    # Third convolutional level
    conv3 = Conv2D(128, (3, 3), activation=activation_function)(inputs)
    gmp3 = GlobalMaxPooling2D()(conv3)
    fc3 = Dense(64, activation=activation_function)(gmp3)

    # Concatenate layers
    # First Concatenation
    concatenate1 = Concatenate()([fc1, fc2])

    # Second Concatenation
    concatenate2 = Concatenate()([concatenate1, fc3])

    # Fully Connected Layers after Concatenation
    fc4 = Dense(128, activation=activation_function)(concatenate2)
    fc5 = Dense(64, activation=activation_function)(fc4)

    # Output layer
    outputs = Dense(num_classes, activation=output_activation)(fc5)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
# Compile the model
def compile_model(model):
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
# Train the model
def train_model(model, train_generator, val_generator, epochs=epochs):
    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)
    return history

In [ ]:
# Evaluate the model
def evaluate_model(model, test_generator):
    results = model.evaluate(test_generator)
    return results

In [ ]:
# Execute and Test the model

train_generator, val_generator, test_generator = preprocess_data(train_dir, val_dir, test_dir)
model = create_model(input_shape, num_classes)
compile_model(model)
history = train_model(model, train_generator, val_generator)
test_results = evaluate_model(model, test_generator)